### For perfect

In [1]:
import os
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

figsuplix = "pdf"

In [2]:
def is_Exist_file(path):
    import os
    if os.path.exists(path):
        os.remove(path)


def mkdir(path):
    import os
    path = path.strip()  # 去除首位空格
    path = path.rstrip("\\")  # 去除尾部 \ 符号
    isExists = os.path.exists(path)  # 判断路径是否存在
    # 判断结果
    if not isExists:
        os.makedirs(path)  # 如果不存在则创建目录
        print(path + ' 创建成功')
    else:
        print(path + ' 目录已存在')  # 如果目录存在则不创建，并提示目录已存在

In [3]:
## mutation 坐标转换
#####################################################################################
## 反向互补
def reverseComplement(seq):
    """
     生成反向互补序列
     :param seq:
     :return:revComSeq
     """
    ATCG_dict = {'A': 'T', 'T': 'A', 'C': 'G', 'G': 'C', 'a': 't', 't': 'a', 'c': 'g', 'g': 'c', 'N': 'N'}
    revComSeq = ''
    for i in seq:
        revComSeq = ATCG_dict[i] + revComSeq
    return revComSeq


## 转换核心函数
def transformer_core(mut, index_dict):
    if 'M' in mut:
        inf, sup = int(mut.split(':')[0][1:]), int(mut.split(':')[1][:3])
        bn, an = mut.split('_')[1].split(':')[0], mut.split('_')[1].split(':')[1]
        new_inf, new_sup = index_dict[inf], index_dict[sup]
        new_bn, new_an = reverseComplement(bn), reverseComplement(an)
        return '%s%s:%sM_%s:%s'%(mut[0], new_inf, new_sup, new_bn, new_an)
    elif 'I' in mut:
        new_inser_site = index_dict[int(mut.split('_')[0][1:-1])]
        new_inser_nucles = reverseComplement(mut.split('_')[-1])
        return '%s%sI_%s'%(mut[0], new_inser_site, new_inser_nucles)
    elif 'D' in mut:
        inf, sup = int(mut.split(':')[0][1:]), int(mut.split(':')[-1][:-1])
        new_inf, new_sup = index_dict[inf], index_dict[sup]
        return '%s%s:%sD'%(mut[0], new_inf, new_sup)
    else:
        return mut


## 解析 all_mutation & target_mutation 信息
def parse_mutation_information(mutation):
    '''
    mutation = 'X0+Y179:180D.Y186:186M_C:A+Z196I_T'
    # mutation = 'X0+Y0+Z0'
    muts = parse_mutation_information(mutation)
    '''
    import re
    mut_list = re.split('\+|\.', mutation)
    mut_dict = {'X': [], 'Y': [], 'Z': []}
    for mut in mut_list:
        mut_dict[mut[0]].append(mut)
    return mut_dict


## 调整突变坐标的参考系：以 gRNASeq 为基
def adjust_off_target_mutaion_reference_system(mutation):
    '''
    mutation = 'X164I_T+Y178I_C+Z0'
    mutation = 'X164:164D+Y187I_G+Z199:199M_C:T'
    mutation = 'X0+Y0+Z0'
    adjust_off_target_mutaion_reference_system(mutation)
    '''
    ## 长序列与短序列对应字典
    index_dict = {}
    for i in range(20):
        index_dict[125 + i] = -19 + i
    for i in range(63):
        index_dict[145 + i] = 1 + i
    for i in range(20):
        index_dict[208 + i] = 64 + i
    ## 解析 all_mutation & target_mutation 信息
    mut_dict = parse_mutation_information(mutation)
    new_mut_dict = {'X': '', 'Y': '', 'Z': ''}
    for label, mut_list in mut_dict.items():
        a_list = []
        for mut in mut_list:
            new_mut = transformer_core(mut, index_dict)
            a_list.append(new_mut)
        a_list.sort(reverse=False)
        new_mut_dict[label] = '.'.join(a_list)
    new_list = []
    for label in ['X', 'Y', 'Z']:
        new_list.append(new_mut_dict[label])
    return '+'.join(new_list)
#####################################################################################

In [4]:
## up, core, down 区域内突变核苷酸总长度
def region_mutation_length(mutation):
    mut_list = mutation.split(".")
    mut_length = 0
    for mut in mut_list:
        if 'M' in mut:
            mut_length += len(mut.split(":")[-1])
        elif 'I' in mut:
            mut_length += len(mut.split("_")[-1])
        elif 'D' in mut:
            inf, sup = int(mut.split(':')[0][1:]), int(mut.split(':')[-1][:-1])
            delt_length = sup - inf + 1
            mut_length += delt_length
        else:
            pass
    return mut_length


def mutation_region_splitting(data):
    ## step 1: mutaion 区域划分
    data['new_mutation'] = data['all_mutation'].apply(lambda x: adjust_off_target_mutaion_reference_system(x))
    ## up, core, down 区域内突变类型
    data['up_mutation'] = data['new_mutation'].apply(lambda x: x.split('+')[0])
    data['core_mutation'] = data['new_mutation'].apply(lambda x: x.split('+')[1])
    data['down_mutation'] = data['new_mutation'].apply(lambda x: x.split('+')[2])
    ## up, core, down 区域内间断突变的个数
    data['up_mut_num'] = data['up_mutation'].apply(lambda x: len(x.split('.')) if x != 'X0' else 0)
    data['core_mut_num'] = data['core_mutation'].apply(lambda x: len(x.split('.')) if x != 'Y0' else 0)
    data['down_mut_num'] = data['down_mutation'].apply(lambda x: len(x.split('.')) if x != 'Z0' else 0)
    ## up, core, down 区域内突变核苷酸总长度
    data['up_mut_length'] = data['up_mutation'].apply(lambda x: region_mutation_length(x))
    data['core_mut_length'] = data['core_mutation'].apply(lambda x: region_mutation_length(x))
    data['down_mut_length'] = data['down_mutation'].apply(lambda x: region_mutation_length(x))
    ############################################################
    print("\nFirst: 统计 X0+Y0+Z0、XA+Y0+Z0、X0+YA+Z0、X0+Y0+ZA、XA+YA+Z0、XA+Y0+ZA、X0+YA+ZA、XA+YA+ZA 的类型占比")
    data_000 = data.loc[(data['up_mut_num']==0) & (data['core_mut_num']==0) & (data['down_mut_num']==0), :]
    data_100 = data.loc[(data['up_mut_num']!=0) & (data['core_mut_num']==0) & (data['down_mut_num']==0), :]
    data_010 = data.loc[(data['up_mut_num']==0) & (data['core_mut_num']!=0) & (data['down_mut_num']==0), :]
    data_001 = data.loc[(data['up_mut_num']==0) & (data['core_mut_num']==0) & (data['down_mut_num']!=0), :]
    data_110 = data.loc[(data['up_mut_num']!=0) & (data['core_mut_num']!=0) & (data['down_mut_num']==0), :]
    data_101 = data.loc[(data['up_mut_num']!=0) & (data['core_mut_num']==0) & (data['down_mut_num']!=0), :]
    data_011 = data.loc[(data['up_mut_num']==0) & (data['core_mut_num']!=0) & (data['down_mut_num']!=0), :]
    data_111 = data.loc[(data['up_mut_num']!=0) & (data['core_mut_num']!=0) & (data['down_mut_num']!=0), :]
    ## count
    count_dict = {}
    count_000 = data_000.shape[0]
    count_100, count_010, count_001 = data_100.shape[0], data_010.shape[0], data_001.shape[0]
    count_110, count_101, count_011 = data_110.shape[0], data_101.shape[0], data_011.shape[0]
    count_111 = data_111.shape[0]
    count_dict['X0+Y0+Z0'] = count_000
    count_dict['XA+Y0+Z0'] = count_100
    count_dict['X0+YA+Z0'] = count_010
    count_dict['X0+Y0+ZA'] = count_001
    count_dict['XA+YA+Z0'] = count_110
    count_dict['XA+Y0+ZA'] = count_101
    count_dict['X0+YA+ZA'] = count_011
    count_dict['XA+YA+ZA'] = count_111
    print("X0+Y0+Z0:", count_000)
    print("XA+Y0+Z0:", count_100)
    print("X0+YA+Z0:", count_010)
    print("X0+Y0+ZA:", count_001)
    print("XA+YA+Z0:", count_110)
    print("XA+Y0+ZA:", count_101)
    print("X0+YA+ZA:", count_011)
    print("XA+YA+ZA:", count_111)
    count_sum = data.shape[0]
    print("Distribution of count ratio:", round(count_000/count_sum, 3), 
                        round(count_100/count_sum, 3), round(count_010/count_sum, 3), round(count_001/count_sum, 3), 
                        round(count_110/count_sum, 3), round(count_101/count_sum, 3), round(count_011/count_sum, 3), 
                        round(count_111/count_sum, 3))
    return (data_000, data_100, data_010, data_001, data_110, data_101, data_011, data_111, count_dict)



In [5]:
## 1.1 on-target 编辑效率分析
def distplot_on_target_efficiency(data_000, cell_line, replicate, save_dir):
    import seaborn as sns
    import matplotlib.pyplot as plt
    sns.set(color_codes=True)
    fig, ax = plt.subplots(1,1, figsize=(6, 5))
    # 为坐标轴设置名称
    colors = ['slategrey','lightsteelblue','cornflowerblue','royalblue','darkorange','lightgreen',
         'green','r','teal','blue','slateblue','darkslateblue',
         'maroon','blueviolet','indigo','darkorchid','mediumorchid','thistle',
         'plum','violet','purple','m','lightseagreen','magenta',
         'orchid','chartreuse','deeppink','hotpink','palevioletred','pink']
    sns.distplot(data_000['off-target_eff'], hist=False, color='purple')
    plt.xlabel("on-target efficiency", fontsize=12, weight='bold')
    plt.ylabel("Density",fontsize=12, weight='bold')
    title = "%s-%s on-target editing efficiency"%(cell_line, replicate)
    plt.title(title, fontsize=12, weight='bold')
    savefig_path = '%s/%s.%s'%(save_dir, title, figsuplix)
    plt.savefig(savefig_path, dpi=300, bbox_inches = 'tight')
    plt.show()

### For mismatch

In [6]:
import os
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [7]:
########################################################
def is_Exist_file(path):
    import os
    if os.path.exists(path):
        os.remove(path)


def mkdir(path):
    import os
    path = path.strip()  # 去除首位空格
    path = path.rstrip("\\")  # 去除尾部 \ 符号
    isExists = os.path.exists(path)  # 判断路径是否存在
    # 判断结果
    if not isExists:
        os.makedirs(path)  # 如果不存在则创建目录
        print(path + ' 创建成功')
    else:
        print(path + ' 目录已存在') 

#### 1. distribution of off-target mismatch

###### 1.1 distribution of the mismatch number

In [8]:
## core 区域内 mutation 的长度
def get_off_target_mismatch_number(data_010):
    core_mut_length_dict = {}
    mut_length_list = list(data_010['core_mut_length'].unique())
    for mut_length in mut_length_list: 
        templ = data_010.loc[data_010['core_mut_length']==mut_length, :].shape[0]
        core_mut_length_dict['core_mut_length_' + str(mut_length)] = round(templ/data_010.shape[0], 3)
    print("\ncore mutation length:")
    print(core_mut_length_dict)
    return core_mut_length_dict


## plot distribution of off-target mismatch number
def plot_distribution_off_target_mismacth_num(core_mut_length_dict, save_dir):
    data_list = []
    labels = []
    keys = list(core_mut_length_dict.keys())
    keys.sort(reverse=False)
    for k in keys:
        v = core_mut_length_dict[k]
        data_list.append(round(v*100, 2))
        labels.append(k.split('_')[-1])

    ## 柱状图
    import matplotlib.pyplot as plt
    # 设置默认绘图风格
    plt.style.use("seaborn-white")  
    fig, ax = plt.subplots(1,1, figsize=(6, 4))

    plt.bar(range(len(data_list)), data_list)
    ## 坐标轴不可见
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ## xlabel, ylabel
    plt.xlabel("Mismatch length of Target+PAM Region", fontsize=12, weight='bold')
    plt.ylabel("Ratio (%)", fontsize=12, weight='bold')
    ## xticks
    plt.xticks(range(len(labels)), labels, fontsize=12, weight='bold')
    ## text
    ## title
    title = 'distribution of off-target mismatch number'
    plt.title(title, fontsize=12, weight='bold')
    ## save
    mkdir(save_dir)
    savefig_path = save_dir + '/%s.%s'%(title, figsuplix)
    plt.savefig(savefig_path, dpi=300, bbox_inches = 'tight')
    plt.show()

###### 1.2 distribution of off-target mismatch among Target + PAM regions

In [9]:
## 确定 mutation site 的上下界限
def mutation_site_range(mut_site):
    if 'M' in mut_site:
        mutk = mut_site.split("_")[0].split(':')
        inf, sup = int(mutk[0][1:]), int(mutk[1][:-1])
    elif 'I' in mut_site:
        mut = mut_site.split("_")[0]
        inf, sup = int(mut[1:-1]), int(mut[1:-1])
    elif 'D' in mut_site: ## deletion
        mut = mut_site.split(":")
        inf, sup = int(mut[0][1:]), int(mut[1][:-1])
    else: ## X0/Y0/Z0
        inf, sup = 0, 0
    return (inf, sup)


## 计算 core 区域 mut_num == x 的位置分布
def mutation_length_x_position_distribution(data):
    pos_dict = {}
    for i in range(0, 63, 1):
        pos_dict[i] = 0
    for index, row in data.iterrows():
        mut = row['core_mutation']
        inf, sup = mutation_site_range(mut)
        for i in range(inf, sup + 1, 1):
            pos_dict[i] += 1
    sum_count = sum(list(pos_dict.values()))
    pos_ratio_dict = {}
    for pos, count in pos_dict.items():
        pos_ratio_dict[pos] = count/sum_count
    return pos_ratio_dict


## plot distribution of off-target mismatch among Target + PAM regions
def plot_distribution_off_target_mismatch_with_positions(data, save_dir):
    for x in range(1, 7, 1):
        data_x = data.loc[data['core_mut_length']==x, :]
        data_x.reset_index(drop=True, inplace=True)
        pos_ratio_dict = mutation_length_x_position_distribution(data_x)
        print("YA 内的 mismatch number == x  的位置分布:")
        # print(pos_ratio_dict)
        ## 柱状图展示 mutation 的位置分布
        labels = []
        data_list = []
        for k in range(21, 44):
            if k == 41:
                labels.append("N")
            elif (k == 42) | (k == 43):
                labels.append("G")
            else:
                labels.append(k-20)
            data_list.append(pos_ratio_dict[k])
        ############################################################
        ## 柱状图
        import matplotlib.pyplot as plt
        # 设置默认绘图风格
        plt.style.use("seaborn-white")  
        fig, ax = plt.subplots(1,1, figsize=(12, 4))

        plt.bar(range(len(data_list)), data_list, color='lightseagreen')
        ## 坐标轴不可见
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ## xlabel, ylabel
        plt.xlabel("Target+PAM Region", fontsize=12, weight='bold')
        plt.ylabel("Ratio", fontsize=12, weight='bold')
        ## xticks
        plt.xticks(range(len(labels)), labels, fontsize=12, weight='bold')
        ## text
        ## title
        title = 'position distribution of off-target mismatch_number-%s'%(x)
        plt.title(title, fontsize=12, weight='bold')
        ## save
        mkdir(save_dir)
        savefig_path = save_dir + '/%s.%s'%(title, figsuplix)
        plt.savefig(savefig_path, dpi=300, bbox_inches = 'tight')
        plt.show()
############################################################

###### 1.3. distribution of substitution types

In [10]:
## step 2.3: substitution 的类型
## core region 内的核苷酸替换类型
def obtain_substitution_type(data):
    sub_dict = {'A->C': 0, 'A->G': 0, 'A->T': 0, 
                'C->A': 0, 'C->G': 0, 'C->T': 0, 
                'G->A': 0, 'G->C': 0, 'G->T': 0, 
                'T->A': 0, 'T->C': 0, 'T->G': 0}
    for index, row in data.iterrows():
        mutation = row['core_mutation']
        import re
        mut_list = re.split('\+|\.', mutation)
        for mut in mut_list:
            if 'M' in mut:
                subs = mut.split('_')[-1].split(':')
                ori_n, src_n = subs[0], subs[1]
                for index, n in enumerate(ori_n):
                    sub = '%s->%s'%(n, src_n[index])
                    sub_dict[sub] += 1
            else:
                pass
    ## ratio
    sum_c = sum(list(sub_dict.values()))
    new_sub_dict = {}
    for k, v in sub_dict.items():
        new_sub_dict[k] = v/sum_c
    return new_sub_dict


####################################################################
## mismatch length == x
def plot_distribution_substitution_types(data, save_dir):
    for x in range(1, 7, 1): 
        data_x = data.loc[data['core_mut_length']==x, :]
        data_x.reset_index(drop=True, inplace=True)
        sub_dict = obtain_substitution_type(data_x)
        print('core region 内的核苷酸替换类型:')
        # print(sub_dict)
        labels = []
        data_list = []
        for label, v in sub_dict.items():
            labels.append(label)
            data_list.append(v)

        ####################################################################
        ## 柱状图
        import matplotlib.pyplot as plt
        # 设置默认绘图风格
        plt.style.use("seaborn-white")  
        fig, ax = plt.subplots(1,1, figsize=(12, 4))

        plt.bar(range(len(data_list)), data_list, color='slateblue')
        ## 坐标轴不可见
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ## xlabel, ylabel
        plt.xlabel("Target+PAM Region", fontsize=12, weight='bold')
        plt.ylabel("Ratio", fontsize=12, weight='bold')
        ## xticks
        plt.xticks(range(len(labels)), labels, fontsize=12, weight='bold')
        ## text
        ## title
        title = 'Distribution of nucleotide substitution type of off-target mismatch_number-%s'%(x)
        plt.title(title, fontsize=12, weight='bold')
        ## save
        mkdir(save_dir)
        savefig_path = save_dir + '/%s.%s'%(title, figsuplix)
        plt.savefig(savefig_path, dpi=300, bbox_inches = 'tight')
        plt.show()
####################################################################

#### 2. Heatmap of off-target single mismatch analysis

In [11]:
## mismatch length == x
## 确定 mutation site 的上下界限
def mutation_site_range(mut_site):
    if 'M' in mut_site:
        mutk = mut_site.split("_")[0].split(':')
        inf, sup = int(mutk[0][1:]), int(mutk[1][:-1])
    elif 'I' in mut_site:
        mut = mut_site.split("_")[0]
        inf, sup = int(mut[1:-1]), int(mut[1:-1])
    else: ## deletion
        mut = mut_site.split(":")
        inf, sup = int(mut[0][1:]), int(mut[1][:-1])
    return (inf, sup)


## get mismatch position 
def get_off_target_position(mutation):
    import re
    mut_list = re.split('\+|\.', mutation)
    mut_list.sort(reverse=False)
    position_t = []
    for mut_site in mut_list:
        inf, sup = mutation_site_range(mut_site)
        for i in range(inf, sup + 1):
            position_t.append(i)
    return tuple(position_t)


## get mismatch substitution type
def get_mismatch_substitution(mutation):
    import re
    mut_list = re.split('\+|\.', mutation)
    mut_list.sort(reverse=False)
    subs = []
    for mut in mut_list:
        if 'M' in mut:
            ori_nucles, des_nucles = mut.split('_')[-1].split(':')[0], mut.split('_')[-1].split(':')[1]
            for i, nucle in enumerate(ori_nucles):
                sub = (nucle, des_nucles[i])
                subs.append(sub)
    return tuple(subs)


## 获得 mismatch length == 1 的某一位置的 heatmap data
def get_plot_mismatch1_heatmap_data(pos, data_010, on_data):
    ## parse position_tuple & substitution 
    data_010_x = data_010.loc[data_010['core_mut_length'] == 1, :]
    data_010_x['position_tuple'] = data_010_x['core_mutation'].apply(lambda x: get_off_target_position(x))
    data_010_x['substitution'] = data_010_x['core_mutation'].apply(lambda x: get_mismatch_substitution(x))
    ## merge off-data & on-data
    on_data.rename(columns={'off-target_eff': 'on-target_eff'}, inplace=True)
    data_010_x = pd.merge(data_010_x, on_data[['sgRNA_name', 'on-target_eff']], how='inner', on='sgRNA_name')
    data_010_x['reltv_eff'] = data_010_x.apply(lambda row: row['off-target_eff']/row['on-target_eff'], axis=1)
    ## get plot data
    position_tuple = (pos,)
    pdata = data_010_x.loc[data_010_x['position_tuple'] == position_tuple, :]
    subs_list = list(pdata['substitution'].unique()) ## 所有可能的替换类型
    stat_count = pd.DataFrame({'A': [0]*4, 'C': [0]*4, 'G': [0]*4, 'T': [0]*4})
    plot_data = pd.DataFrame({'A': [0]*4, 'C': [0]*4, 'G': [0]*4, 'T': [0]*4})
    plot_data.index = ['A', 'C', 'G', 'T']
    stat_count.index = ['A', 'C', 'G', 'T']
    for subs in subs_list:
        for index, sub in enumerate(subs):
            position = position_tuple[index]
            ori_nucle, des_nucle = sub ## 列是原始核苷酸，行是替换的核苷酸
            plot_data.loc[des_nucle, ori_nucle] = pdata.loc[pdata['substitution'] == subs, 'reltv_eff'].median()
            stat_count.loc[des_nucle, ori_nucle] = pdata.loc[pdata['substitution'] == subs, :].shape[0]/data_010_x.shape[0]
    return plot_data, stat_count

In [12]:
## 确定所以值得最大、最小
def set_vmin_max(data_010, on_data, data_id, vmin, vmax, ori_pos=21, des_pos=44):
    for pos in range(ori_pos, des_pos, 1):
        data_tuple = get_plot_mismatch1_heatmap_data(pos, data_010, on_data)
        data = data_tuple[data_id]
        vmin = min([np.min(np.array(data)), vmin])
        vmax = max([np.max(np.array(data)), vmax])
    return vmin, vmax


## plot heatmap
def help_creak_mask(plot_data):
    mask = np.zeros_like(plot_data)
    for i in range(plot_data.shape[0]):
        for j in range(plot_data.shape[1]):
            # print(i, j, plot_data.iloc[i, j])
            if plot_data.iloc[i, j] == 0:
                mask[i, j] = 0
    return mask


## 展示 mismatch 的 position-substitution 的分布
## For position 1 to 21
def plot_multiheatmap_ratio_1(data_010, on_data, cell_line, replicate, suffix, title, save_dir, mask, 
                            vmin=0, vmax=1, data_id=0, cmap="Greys"):
    ## position annotation: transforme
    tf_pos_dict = {}
    for i in range(21, 44, 1):
        if i == 41:
            tf_pos_dict[i] = "N"
        elif (i == 42) | (i == 43):
            tf_pos_dict[i] = "G"
        else:
            tf_pos_dict[i] = i-20
    
    import matplotlib.pyplot as plt
    import matplotlib.gridspec as gridspec
    import seaborn as sns
    sns.set_style("white")
    sns.set(font_scale=0.6) ## 修改 seaborn 字体比例
    fig = plt.figure(figsize=(15, 1.2))
    k = 23
    gs = gridspec.GridSpec(2, k, height_ratios=[1, 1]) ## 多子图构建，2行 k列

    ## 调整子图间的整体布局
    fig.tight_layout() # 调整整体空白
    plt.subplots_adjust(wspace=0.01, hspace=0) # 调整子图间距

    ytick=['A', 'C', 'G', 'T']
    ## start plot: the left
    pos = 21
    data_tuple = get_plot_mismatch1_heatmap_data(pos, data_010, on_data)
    data = data_tuple[data_id]
    ax0 = plt.subplot(gs[0]) ## 确定子图位置
    sns.heatmap(ax=ax0, data=data, mask=mask, vmin=vmin, vmax=vmax, square=True, cbar=False, yticklabels=ytick, cmap=cmap)
    ax0.set_xlabel(tf_pos_dict[pos], fontsize=12, weight='bold')
    for i in range(1, k-3):
        pos += 1
        data_tuple = get_plot_mismatch1_heatmap_data(pos, data_010, on_data)
        data = data_tuple[data_id]
        ax1 = plt.subplot(gs[i]) ## 确定子图位置
        sns.heatmap(ax=ax1, data=data, mask=mask, vmin=vmin, vmax=vmax, square=True, cbar=False, yticklabels=[], cmap=cmap)
        ax1.set_xlabel(tf_pos_dict[pos], fontsize=10, weight='bold')
        if pos == 32:
            plt.title(title, fontsize=10, weight='bold')
    ## end plot: the right N
    sns.set(font_scale=0.5) ## 修改 seaborn 字体比例
    pos += 1
    data_tuple = get_plot_mismatch1_heatmap_data(pos, data_010, on_data)
    data = data_tuple[data_id]
    ax1 = plt.subplot(gs[k-3]) ## 确定子图位置
    sns.heatmap(ax=ax1, data=data, mask=mask, vmin=vmin, vmax=vmax, square=False, 
                cbar=True, cbar_kws={"shrink": 1, 'ticks': [0.,0.04,0.08,0.12]}, 
                yticklabels=[], cmap=cmap)
    ax1.set_xlabel(tf_pos_dict[pos], fontsize=10, weight='bold')
    ## title
    # plt.title(title, fontsize=10, weight='bold')
    mkdir(save_dir)
    savefig_path = save_dir + '/%s.%s'%(title, figsuplix)
    plt.savefig(savefig_path, dpi=300, bbox_inches = 'tight')

    

## 展示 mismatch 的 position-substitution 的分布
## For PAM-GG
def plot_multiheatmap_ratio_2(data_010, on_data, cell_line, replicate, suffix, title, save_dir, mask, 
                            vmin=0, vmax=1, data_id=0, cmap="Greys"):
    ## position annotation: transforme
    tf_pos_dict = {}
    for i in range(21, 44, 1):
        if i == 41:
            tf_pos_dict[i] = "N"
        elif (i == 42) | (i == 43):
            tf_pos_dict[i] = "G"
        else:
            tf_pos_dict[i] = i-20
    
    import matplotlib.pyplot as plt
    import matplotlib.gridspec as gridspec
    import seaborn as sns
    sns.set_style("white")
    sns.set(font_scale=1) ## 修改 seaborn 字体比例
    fig = plt.figure(figsize=(6, 6))
    k = 2
    gs = gridspec.GridSpec(2, k, height_ratios=[1, 1]) ## 多子图构建，2行 k列

    ## 调整子图间的整体布局
    fig.tight_layout() # 调整整体空白
    plt.subplots_adjust(wspace=0.01, hspace=0) # 调整子图间距

    ytick=['A', 'C', 'G', 'T']
    ## start plot: the left
    ## PAM 42-G
    pos = 42
    data_tuple = get_plot_mismatch1_heatmap_data(pos, data_010, on_data)
    data = data_tuple[data_id]
    ax0 = plt.subplot(gs[0]) ## 确定子图位置
    sns.heatmap(ax=ax0, data=data, mask=mask, vmin=vmin, vmax=vmax, square=True, cbar=False, yticklabels=ytick, cmap=cmap)
    ax0.set_xlabel(tf_pos_dict[pos], fontsize=12, weight='bold')
    # PAM 43-G
    pos = 43
    data_tuple = get_plot_mismatch1_heatmap_data(pos, data_010, on_data)
    data = data_tuple[data_id]
    ax1 = plt.subplot(gs[1]) ## 确定子图位置
    sns.heatmap(ax=ax1, data=data, mask=mask, vmin=vmin, vmax=vmax, 
                square=False, cbar=True, cbar_kws={"shrink": 1, 'ticks': [0.05,0.1,0.15, 0.2, 0.25, 0.3]}, 
                yticklabels=[], cmap=cmap)
    ax1.set_xlabel(tf_pos_dict[pos], fontsize=12, weight='bold')
    ## title
    # plt.title(title, fontsize=10, weight='bold')
    mkdir(save_dir)
    savefig_path = save_dir + '/%s.%s'%(title, figsuplix)
    plt.savefig(savefig_path, dpi=300, bbox_inches = 'tight')


## 展示 mismatch 的 relative indel frequence 的分布
def plot_multiheatmap(data_010, on_data, cell_line, replicate, suffix, title, save_dir, mask, 
                      vmax=1, data_id=0, cmap="Greys"):
    ## position annotation: transforme
    tf_pos_dict = {}
    for i in range(21, 44, 1):
        if i == 41:
            tf_pos_dict[i] = "N"
        elif (i == 42) | (i == 43):
            tf_pos_dict[i] = "G"
        else:
            tf_pos_dict[i] = i-20
    
    import matplotlib.pyplot as plt
    import matplotlib.gridspec as gridspec
    import seaborn as sns
    sns.set_style("white")
    sns.set(font_scale=0.6) ## 修改 seaborn 字体比例
    fig = plt.figure(figsize=(15, 1.2))
    k = 23
    gs = gridspec.GridSpec(2, k, height_ratios=[1, 1]) ## 多子图构建，2行 k列

    ## 调整子图间的整体布局
    fig.tight_layout() # 调整整体空白
    plt.subplots_adjust(wspace=0.01, hspace=0) # 调整子图间距

    ytick=['A', 'C', 'G', 'T']
    ## start plot: the left
    pos = 21
    data_tuple = get_plot_mismatch1_heatmap_data(pos, data_010, on_data)
    data = data_tuple[data_id]
    ax0 = plt.subplot(gs[0]) ## 确定子图位置
    sns.heatmap(ax=ax0, data=data, mask=mask, vmax=vmax, square=True, cbar=False, yticklabels=ytick, cmap=cmap)
    ax0.set_xlabel(tf_pos_dict[pos], fontsize=12, weight='bold')
    for i in range(1, k-1):
        pos += 1
        data_tuple = get_plot_mismatch1_heatmap_data(pos, data_010, on_data)
        data = data_tuple[data_id]
        ax1 = plt.subplot(gs[i]) ## 确定子图位置
        sns.heatmap(ax=ax1, data=data, mask=mask, vmax=vmax, square=True, cbar=False, yticklabels=[], cmap=cmap)
        ax1.set_xlabel(tf_pos_dict[pos], fontsize=10, weight='bold')
        if pos == 32:
            plt.title(title, fontsize=10, weight='bold')
    ## end plot: the right
    sns.set(font_scale=0.5) ## 修改 seaborn 字体比例
    pos += 1
    data_tuple = get_plot_mismatch1_heatmap_data(pos, data_010, on_data)
    data = data_tuple[data_id]
    ax1 = plt.subplot(gs[k-1]) ## 确定子图位置
    sns.heatmap(ax=ax1, data=data, mask=mask, vmax=vmax, square=False, 
                cbar=True, cbar_kws={"shrink": 1, 'ticks': [0.2,0.4, 0.6,0.8]}, 
                yticklabels=[], cmap=cmap)
    ax1.set_xlabel(tf_pos_dict[pos], fontsize=10, weight='bold')
    mkdir(save_dir)
    savefig_path = save_dir + '/%s-2.%s'%(title, figsuplix)
    plt.savefig(savefig_path, dpi=300, bbox_inches = 'tight')

In [13]:
## 展示 mismatch 的 position-substitution 的分布
## For position 1 to 21, N, G, G
def plot_multiheatmap_ratio_3(data_010, on_data, cell_line, replicate, suffix, title, save_dir, mask, 
                            vmin=0, vmax=1, data_id=0, cmap="Greys"):
    ## position annotation: transforme
    tf_pos_dict = {}
    for i in range(21, 44, 1):
        if i == 41:
            tf_pos_dict[i] = "N"
        elif (i == 42) | (i == 43):
            tf_pos_dict[i] = "G"
        else:
            tf_pos_dict[i] = i-20
    
    import matplotlib.pyplot as plt
    import matplotlib.gridspec as gridspec
    import seaborn as sns
    sns.set_style("white")
    sns.set(font_scale=0.6) ## 修改 seaborn 字体比例
    fig = plt.figure(figsize=(15, 1.2))
    k = 23
    gs = gridspec.GridSpec(2, k, height_ratios=[1, 1]) ## 多子图构建，2行 k列

    ## 调整子图间的整体布局
    fig.tight_layout() # 调整整体空白
    plt.subplots_adjust(wspace=0.01, hspace=0) # 调整子图间距

    ytick=['A', 'C', 'G', 'T']
    ## start plot: the left
    pos = 21
    data_tuple = get_plot_mismatch1_heatmap_data(pos, data_010, on_data)
    data = data_tuple[data_id]
    ax0 = plt.subplot(gs[0]) ## 确定子图位置
    sns.heatmap(ax=ax0, data=data, mask=mask, vmin=vmin, vmax=vmax, square=True, cbar=False, yticklabels=ytick, cmap=cmap)
    ax0.set_xlabel(tf_pos_dict[pos], fontsize=12, weight='bold')
    # for i in range(1, k-3):
    for i in range(1, k-1):
        pos += 1
        data_tuple = get_plot_mismatch1_heatmap_data(pos, data_010, on_data)
        data = data_tuple[data_id]
        ax1 = plt.subplot(gs[i]) ## 确定子图位置
        sns.heatmap(ax=ax1, data=data, mask=mask, vmin=vmin, vmax=vmax, square=True, cbar=False, yticklabels=[], cmap=cmap)
        ax1.set_xlabel(tf_pos_dict[pos], fontsize=10, weight='bold')
        if pos == 32:
            plt.title(title, fontsize=10, weight='bold')
    ## end plot: the right N
    sns.set(font_scale=0.5) ## 修改 seaborn 字体比例
    pos += 1
    data_tuple = get_plot_mismatch1_heatmap_data(pos, data_010, on_data)
    data = data_tuple[data_id]
    ax1 = plt.subplot(gs[k-1]) ## 确定子图位置
    sns.heatmap(ax=ax1, data=data, mask=mask, vmin=vmin, vmax=vmax, square=False, 
                cbar=True, cbar_kws={"shrink": 1, 'ticks': [0.,0.002,0.004,0.006]}, 
                yticklabels=[], cmap=cmap)
    ax1.set_xlabel(tf_pos_dict[pos], fontsize=10, weight='bold')
    mkdir(save_dir)
    savefig_path = save_dir + '/%s-total.%s'%(title, figsuplix)
    plt.savefig(savefig_path, dpi=300, bbox_inches = 'tight')

### mismatch 编辑效率分布

In [14]:
## step 3: 编辑效率分析 for X0+YA+Z0
## 获得 off-target 的相对编辑效率 data
## off-target data: fdata
## on-target data: odata
def mismatch1_off_target_relative_efficiency(fdata, odata, vmin_oe=0.05):
    on_dict = {} ## on-target
    for index, row in odata.iterrows():
        sgRNA_name = row['sgRNA_name']
        eff = row['off-target_eff']
        on_dict[sgRNA_name] = eff
    fdata = fdata[['sgRNA_name', 'all_mutation', 'core_mutation', 'off-target_eff']]
    ## 计算 relative eff
    fdata['on-target_eff'] = fdata['sgRNA_name'].apply(lambda x: on_dict[x])
    fdata1 = fdata.loc[fdata['on-target_eff']>=vmin_oe, :]
    fdata1['reltv_eff'] = fdata1.apply(lambda row: row['off-target_eff']/row['on-target_eff'], axis=1)
    ## 计算位置
    fdata1['position'] = fdata1['core_mutation'].apply(lambda x: mutation_site_range(x)[0])
    return fdata1


## Get on-target data
def get_on_off_target_data(data_010, on_data):
    x = 1
    data_010_x = data_010.loc[data_010['core_mut_length']==x, :]
    #############################################
    ## 考虑上下游 mutation
    ## on-id
    on_data['X'] = on_data['new_mutation'].apply(lambda x: x.split('+')[0])
    on_data['Z'] = on_data['new_mutation'].apply(lambda x: x.split('+')[2])
    on_data['id'] = on_data.apply(lambda row: row['sgRNA_name'] + '+' + row['X'] + '+' + row['Z'], axis=1)
    ## off-id
    data_010_x['X'] = data_010_x['new_mutation'].apply(lambda x: x.split('+')[0])
    data_010_x['Z'] = data_010_x['new_mutation'].apply(lambda x: x.split('+')[2])
    data_010_x['id'] = data_010_x.apply(lambda row: row['sgRNA_name'] + '+' + row['X'] + '+' + row['Z'], axis=1)
    
    
    ## common id 
    gRNAs = on_data['id'].tolist()
    offgRNAs = data_010_x['id'].tolist()
    common_gRNAs = list(set(gRNAs) & set(offgRNAs))
    print("on-target gRNAs number:", len(set(gRNAs)))
    print("off-target gRNAs number:", len(set(offgRNAs)))
    print("common gRNAs number:", len(set(common_gRNAs)))
    fdata = data_010_x.loc[data_010_x['id'].isin(common_gRNAs), :] ## off-target
    odata = on_data.loc[on_data['id'].isin(common_gRNAs), :] ## on-target
    print("\n===========================================")
    print("off-target.shape:", fdata.shape)
    print("on-target.shape:", odata.shape)
    return (fdata, odata)

In [15]:
## plot
def plot_relative_eff_with_off_target_pos(fdata1, save_dir):
    import matplotlib.pyplot as plt
    import seaborn as sns

    # 设置默认绘图风格
    plt.style.use("seaborn-white")  
    fig, ax = plt.subplots(1,1, figsize=(12, 6))

    order = [(i) for i in range(21, 44)]
    ax = sns.boxplot(x='position', y='reltv_eff', data=fdata1, width=0.4, color='white', 
                     fliersize=0.5, linewidth=0.5, order=order)
    ## 坐标轴不可见
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    # # 添加分布散点图 boxplot with jitter
    # 通过stripplot添加分布散点图，jitter设置数据间距
    ax = sns.stripplot(x='position', y='reltv_eff', data=fdata1, color="orange", jitter=0.1, size=0.8, order=order)
    ## 添加水平和垂直直线
    plt.axhline(y=1, ls="dashed", c="r", linewidth=1)#添加水平直线
    # plt.vlines(19.5, 0, 5, colors = "r", linestyles = "dashed", linewidth=1)
    ## xlabel, ylabel
    plt.ylabel("Relative editing frequency \nto on-target efficiency", fontsize=12, weight='bold')
    plt.xlabel("Target + PAM Position", fontsize=12, weight='bold')
    plt.ylim(0, 2.3)
    ## xticks
    xticks = list(range(1, 21, 1)) + ['N', 'G', 'G']
    plt.xticks(range(23), xticks, fontsize=10, weight='bold')
    title = 'off-target relative editing efficiency of mismatch number 1'
    plt.title(title, fontsize=12, weight='bold')
    # save_dir = './%s_%s/M4_%s_filtered/pics/%s'%(cell_line, replicate, suffix, 'mismatch')
    mkdir(save_dir)
    savefig_path = save_dir + '/%s.%s'%(title, figsuplix)
    plt.savefig(savefig_path, dpi=300, bbox_inches = 'tight')
    plt.show()

### Relative editing efficiency of different mismach number

In [16]:
## 不同 mismatch 长度 off-target 的相对编辑效率
def off_target_relative_efficiency_for_mismatch_length(off_data, on_data, on_eff_cutoff=0.05):
    import copy
    con_data = copy.deepcopy(on_data)
    coff_data = copy.deepcopy(off_data)
    ## on target data
    con_data = con_data.loc[(con_data['all_mutation'] == 'X0+Y0+Z0') & (con_data['off-target_eff']>=on_eff_cutoff), :]
    print("on_data.shape:", con_data.shape)
    on_dict = {}
    for index, row in con_data.iterrows():
        on_dict[row['sgRNA_name']] = row['off-target_eff']
    ## off target data
    coff_data = coff_data.loc[coff_data['sgRNA_name'].isin(list(on_dict.keys())), 
                           ['sgRNA_name', 'all_mutation', 'core_mut_length', 'off-target_eff']]
    coff_data['on-target_eff'] = coff_data['sgRNA_name'] .apply(lambda x: on_dict[x])
    coff_data['reltv_eff'] = coff_data.apply(lambda row: row['off-target_eff']/row['on-target_eff'], axis=1)
    coff_data.reset_index(drop=True, inplace=True)
    return coff_data


## 随机选择长度从1 到 8 的 mismach， 计算相对编辑效率
def get_random_mismatch_length_data(off_data, count):
    import random
    stat_data_list = []
    for mut_length in range(1, 8):
        temp = off_data.loc[off_data['core_mut_length'] == mut_length, :]
        random_index = random.sample(list(temp.index), count)
        ## DataFrame
        stat_data = pd.DataFrame()
        stat_data['reltv_eff'] = temp.loc[random_index, :]['reltv_eff'].tolist()
        stat_data['mut_length'] = mut_length
        stat_data_list.append(stat_data)
    stat_data = pd.concat(stat_data_list, axis=0)
    return stat_data


## plot
def plot_mismatch_number_on_reltv_eff(stat_data, save_dir):
    import matplotlib.pyplot as plt
    import seaborn as sns

    # 设置默认绘图风格
    plt.style.use("seaborn-white")  
    fig, ax = plt.subplots(1,1, figsize=(6, 3))

    order = [i for i in range(1, 8)]
    ax = sns.boxplot(x='mut_length', y='reltv_eff', data=stat_data, width=0.4, color='white', 
                     fliersize=0.5, linewidth=0.5, order=order)
    ## 坐标轴不可见
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ## 添加分布散点图 boxplot with jitter
    ## 通过stripplot添加分布散点图，jitter设置数据间距
    ax = sns.stripplot(x='mut_length', y='reltv_eff', data=stat_data, color="orange", jitter=0.1, size=0.8, order=order)
    plt.ylabel("Relative editing frequency \nto on-target efficiency", fontsize=12, weight='bold')
    plt.xlabel("Number of mismatches", fontsize=12, weight='bold')
    plt.ylim(-0.08, 1.58)
    ## xticks
    xticks = order
    plt.xticks(range(len(xticks)), xticks, fontsize=10, weight='bold')
    title = 'The effect of mismatch number in Target PAM Region on editing efficiency'
    plt.title(title, fontsize=12, weight='bold')
    mkdir(save_dir)
    savefig_path = save_dir + '/%s.%s'%(title, figsuplix)
    stat_data.to_csv(save_dir + "/%s.csv"%(title), index=False)
    s = stat_data.loc[(stat_data['mut_length']==1), :].shape[0]
    a = stat_data.loc[(stat_data['reltv_eff']>=0.5) & (stat_data['mut_length']==1), :].shape[0]
    print(a/s)
    plt.savefig(savefig_path, dpi=300, bbox_inches = 'tight')
    plt.show()
########################################################

### Summary

In [17]:
## load data
def Obtain_mismatch_data(read_data_dir, mut_type):
    if cell_line == 'H1':
        reads_num, barcode_num = 400, 10
    else:
        reads_num, barcode_num = 200, 10
    ## read
    data_path = '%s/off-target.%s.accurate.offseq'%(read_data_dir, mut_type)
    data = pd.read_csv(data_path, sep='\t')
    data = data.loc[(data['reads_num']>=reads_num) & (data['barcode_num']>=barcode_num), :]
    data.reset_index(drop=True, inplace=True)
    print('columns:', data.columns.tolist())
    print(data.shape)

    ## data 根据 up, core, down 区域突变与否划分数据集 
    data_000, data_100, data_010, data_001, data_110, data_101, data_011, data_111, count_dict = mutation_region_splitting(data)
    return (data, data_010, reads_num, barcode_num)


## obtain on target data
def obtaim_on_target_data(read_data_dir, reads_num, barcode_num):
    on_data_path = '%s/off-target.perfect.accurate.offseq'%(read_data_dir)
    on_data = pd.read_csv(on_data_path, sep='\t')
    on_data = on_data.loc[(on_data['all_mutation']=='X0+Y0+Z0') & (on_data['off-target_eff']>=0.05), :]
    on_data = on_data.loc[(on_data['reads_num']>=reads_num) & (on_data['barcode_num']>=barcode_num), :]
    on_data.reset_index(drop=True, inplace=True)
    return on_data


## summary_plot
def plot_all_mismatch(cell_line, replicate, data, data_010, on_data):
    import copy 
    print("\n## plot distribution of off-target mismatch number")
    core_mut_length_dict = get_off_target_mismatch_number(data_010)
    save_dir = './%s/off-target/FigSupp-distribution of off-target mismatch num/%s-%s'%(figsuplix, cell_line, replicate)
    mkdir(save_dir)
    plot_distribution_off_target_mismacth_num(core_mut_length_dict, save_dir)
    ##################################################
    
    ## plot distribution of off-target mismatch among Target + PAM regions
    print("\n## plot distribution of off-target mismatch among Target + PAM regions")
    save_dir = './%s/off-target/FigSupp-distribution of mismatch with position/%s-%s'%(figsuplix, cell_line, replicate)
    mkdir(save_dir)
    plot_distribution_off_target_mismatch_with_positions(data, save_dir)
    ##################################################
    
    ## plot distribution of substitution types
    print("\n## plot distribution of substitution types")
    save_dir = './%s/off-target/FigSupp-distribution of substitution types/%s-%s'%(figsuplix, cell_line, replicate)
    mkdir(save_dir)
    plot_distribution_substitution_types(data, save_dir)
    ##################################################
    
    ## data preparation for relative off-target efficiency
    print("\n## 对 mismatch position & substitution 的分布分析 ")
    on_data_copy = copy.deepcopy(on_data)
    pos = 21
    # for pos in range(21, 44):
    print('\nposition:', pos)
    print("===============================")
    plot_data, stat_count = get_plot_mismatch1_heatmap_data(pos, data_010, on_data_copy)
    mask = help_creak_mask(plot_data)
    ## 对 mismatch position & substitution 的分布分析 
    ## type distribution
    save_dir = './%s/off-target/Fig20-heatmap of distribution of substitution/%s-%s'%(figsuplix, cell_line, replicate)
    mkdir(save_dir)
    title0 = 'Ratio distribution of substitution of off-target mismatch-1'
    plot_multiheatmap_ratio_3(data_010, on_data, cell_line, replicate, suffix, title0, save_dir, mask, 
                          vmin=0, vmax=0.008, data_id=1, cmap="Greys")
    ## relative efficiency
    ## 对 mismatch position & substitution 的分布分析
    title1 = 'off-target relative efficiency of substitution of mismatch-1'
    save_dir = './%s/off-target/Fig21-heatmap of distribution of relative efficiency/%s-%s'%(figsuplix, cell_line, replicate)
    mkdir(save_dir)
    plot_multiheatmap(data_010, on_data_copy, cell_line, replicate, suffix, title1, save_dir, mask, 
                      vmax=1, data_id=0, cmap="Greys")
    ##################################################
    
    ## 获得 off-target 的相对编辑效率 data
    print("\n## plot_relative_eff_with_off_target_pos")
    on_data.rename(columns={"on-target_eff": "off-target_eff"}, inplace=True)
    on_data_copy = copy.deepcopy(on_data)
    fdata, odata = get_on_off_target_data(data_010, on_data_copy)
    fdata1 = mismatch1_off_target_relative_efficiency(fdata, odata, vmin_oe=0.05)
    ## plot_relative_eff_with_off_target_pos
    save_dir = './%s/off-target/Fig21-heatmap of distribution of relative efficiency/%s-%s'%(figsuplix, cell_line, replicate)
    mkdir(save_dir)
    plot_relative_eff_with_off_target_pos(fdata1, save_dir)
    ##################################################
    
    ## 不同 mismatch 长度 off-target 的相对编辑效率
    print("\n## plot Relative editing efficiency of different mismach number")
    on_data_copy = copy.deepcopy(on_data)
    off_data = off_target_relative_efficiency_for_mismatch_length(data_010, on_data_copy, on_eff_cutoff=0.05)
    print(off_data.shape)
    ## 随机选择 count 个长度从1 到 8 的 mismach， 计算相对编辑效率
    count = 500
    stat_data = get_random_mismatch_length_data(off_data, count)
    print(stat_data.shape)
    save_dir = './%s/off-target/Fig22-effect of mismatch number/%s-%s'%(figsuplix, cell_line, replicate)
    mkdir(save_dir)
    plot_mismatch_number_on_reltv_eff(stat_data, save_dir)
    return off_data